In [1]:
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import adapt, register_adapter, AsIs
from sqlalchemy import create_engine, text
import getpass
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
database = 'f24t03'
user = 'jsmm8'
password = getpass.getpass("Type password and hit enter: ")

Type password and hit enter: ········


In [ ]:
try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(dbname=database, user=user, password=password, host='pgsql')
    cursor = connection.cursor()

    # Execute a query to get all table names
    cursor.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public';
    """)
    
    # Fetch all results
    tables = cursor.fetchall()

    # Print the table names
    print("Tables in the database:")
    for table in tables:
        print(table[0])

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()

Tables in the database:
ethnicity
year_built
education_levels
tenure_by_bedrooms
tenure_by_kitchen
tenure_by_plumbing
education_data_processed
rooms
imputed_median_house_value
demographics_master
education_categories
year_structure_built
race_data_processed
mortgage
tenure_by_rooms
merged_new_house_data
housing_stats_data
bedrooms
educational_attainment
employment_status
geo_mobility
home_value
house_heating_fuel
race_migration_data
household_income
internet_subscription
income_to_poverty_ratio
median_home_value
median_household_income
median_rooms_by_tenure
occupancy_status
plumbing_facilities
poverty_status
mortgage_status
housing_pct_change_stats_data
income_pct_change
income_data_processed


In [ ]:
try:
    conn = psycopg2.connect(database=database,
                            user=user,
                            host='pgsql',
                            password=password)
    print("I am able to connect to the database")
except Exception as e:
    print("I am unable to connect to the database:", e)
    raise

tables = [
    "employment_status",
    "year_structure_built",
    "tenure_by_bedrooms",
    "tenure_by_kitchen",
    "tenure_by_plumbing",
    "median_rooms_by_tenure",
    "mortgage",
    "income_to_poverty_ratio",
]

# Dictionary to store dataframes for each table
dataframes = {}

try:
    for table in tables:
        query = f"SELECT * FROM {table};"
        dataframes[table] = pd.read_sql(query, conn)  # Directly storing DataFrame in the dictionary
        print(f"Data extracted successfully from {table}!")
except Exception as e:
    print("Error extracting data:", e)
finally:
    # Close the connection
    conn.close()

I am able to connect to the database
Data extracted successfully from employment_status!
Data extracted successfully from year_structure_built!
Data extracted successfully from tenure_by_bedrooms!
Data extracted successfully from tenure_by_kitchen!
Data extracted successfully from tenure_by_plumbing!
Data extracted successfully from median_rooms_by_tenure!
Data extracted successfully from mortgage!
Data extracted successfully from income_to_poverty_ratio!


In [5]:
from functools import reduce

try:
    # Using reduce to merge all DataFrames in the dictionary on the common columns
    merged_data = reduce(
        lambda left, right: pd.merge(
            left, right, 
            on=['geo_id', 'block_group', 'census_tract', 'county', 'state', 'year'], 
            how='inner'
        ), 
        dataframes.values()
    )
    print("All datasets merged successfully!")
except Exception as e:
    print("Error merging datasets:", e)

All datasets merged successfully!


In [6]:
merged_data.columns

Index(['geo_id', 'block_group', 'census_tract', 'county', 'state',
       'est_total_x', 'moe_total_x', 'est_tot_in_labor_force',
       'moe_tot_in_labor_force', 'est_tot_civilian',
       ...
       'est_100_to_124', 'moe_100_to_124', 'est_125_to_149', 'moe_125_to_149',
       'est_150_to_184', 'moe_150_to_184', 'est_185_to_199', 'moe_185_to_199',
       'est_200_plus', 'moe_200_plus'],
      dtype='object', length=164)

In [7]:
merged_new_data = merged_data.loc[:, ~merged_data.columns.str.startswith('moe_')]

In [8]:
merged_new_data = merged_new_data.loc[:, ~merged_new_data.columns.str.startswith('est_rent_')]

In [9]:
merged_new_data.columns

Index(['geo_id', 'block_group', 'census_tract', 'county', 'state',
       'est_total_x', 'est_tot_in_labor_force', 'est_tot_civilian',
       'est_tot_civilian_labor_employed', 'est_tot_civilian_unemployed',
       'est_tot_armed_forces', 'est_total_not_in_labor_force', 'year',
       'est_total_y', 'est_own_occ_x', 'est_own_occ_2020_plus',
       'est_own_occ_2010_19', 'est_own_occ_2000_09', 'est_own_occ_1990_99',
       'est_own_occ_1980_89', 'est_own_occ_1970_79', 'est_own_occ_1960_69',
       'est_own_occ_1950_59', 'est_own_occ_1940_49', 'est_own_occ_1939_plus',
       'est_total_x', 'est_own_occ_y', 'est_own_occ_0br', 'est_own_occ_1br',
       'est_own_occ_2br', 'est_own_occ_3br', 'est_own_occ_4br',
       'est_own_occ_5br_plus', 'est_total_y', 'est_own_occ_x',
       'est_own_occ_kit_complete', 'est_own_occ_kit_lack', 'est_total_x',
       'est_own_occ_y', 'est_own_occ_plumb_complete', 'est_own_occ_plumb_lack',
       'est_median_rooms_total', 'est_median_rooms_owner',
       'es

In [10]:
merged_new_data.isnull().sum()[merged_new_data.isnull().sum() > 0]

est_median_rooms_total           81
est_median_rooms_owner          535
est_median_rooms_renter        2835
est_home_equity_only          15093
est_both_mortgage_and_loan    15093
est_no_mortgage_or_loan       15093
dtype: int64

In [11]:
columns_to_drop = ['est_home_equity_only','est_both_mortgage_and_loan','est_no_mortgage_or_loan', 'est_median_rooms_renter', 
                   'est_only_home_equity_loan', 'est_home_equity_loan_only']

merged_new_house_data = merged_new_data.drop(columns=columns_to_drop)

In [12]:
merged_new_house_data.columns

Index(['geo_id', 'block_group', 'census_tract', 'county', 'state',
       'est_total_x', 'est_tot_in_labor_force', 'est_tot_civilian',
       'est_tot_civilian_labor_employed', 'est_tot_civilian_unemployed',
       'est_tot_armed_forces', 'est_total_not_in_labor_force', 'year',
       'est_total_y', 'est_own_occ_x', 'est_own_occ_2020_plus',
       'est_own_occ_2010_19', 'est_own_occ_2000_09', 'est_own_occ_1990_99',
       'est_own_occ_1980_89', 'est_own_occ_1970_79', 'est_own_occ_1960_69',
       'est_own_occ_1950_59', 'est_own_occ_1940_49', 'est_own_occ_1939_plus',
       'est_total_x', 'est_own_occ_y', 'est_own_occ_0br', 'est_own_occ_1br',
       'est_own_occ_2br', 'est_own_occ_3br', 'est_own_occ_4br',
       'est_own_occ_5br_plus', 'est_total_y', 'est_own_occ_x',
       'est_own_occ_kit_complete', 'est_own_occ_kit_lack', 'est_total_x',
       'est_own_occ_y', 'est_own_occ_plumb_complete', 'est_own_occ_plumb_lack',
       'est_median_rooms_total', 'est_median_rooms_owner', 'est_total

In [13]:
merged_new_house_data.columns = [col if i not in [5, 13, 14] else ('est_pop_16_and_above' if i == 5 else 'est_occ' if i == 13 else 'est_own_occ') for i, col in enumerate(merged_new_house_data.columns)]

In [14]:
merged_new_house_data.dtypes

geo_id                              object
block_group                         object
census_tract                        object
county                              object
state                               object
est_pop_16_and_above                 int64
est_tot_in_labor_force               int64
est_tot_civilian                     int64
est_tot_civilian_labor_employed      int64
est_tot_civilian_unemployed          int64
est_tot_armed_forces                 int64
est_total_not_in_labor_force         int64
year                                 int64
est_occ                              int64
est_own_occ                          int64
est_own_occ_2020_plus                int64
est_own_occ_2010_19                  int64
est_own_occ_2000_09                  int64
est_own_occ_1990_99                  int64
est_own_occ_1980_89                  int64
est_own_occ_1970_79                  int64
est_own_occ_1960_69                  int64
est_own_occ_1950_59                  int64
est_own_occ

In [15]:
merged_new_house_data = merged_new_house_data.drop(columns=['est_total_x', 'est_own_occ_y', 'est_total_y', 'est_own_occ_x'])

In [16]:
merged_new_house_data.dtypes

geo_id                              object
block_group                         object
census_tract                        object
county                              object
state                               object
est_pop_16_and_above                 int64
est_tot_in_labor_force               int64
est_tot_civilian                     int64
est_tot_civilian_labor_employed      int64
est_tot_civilian_unemployed          int64
est_tot_armed_forces                 int64
est_total_not_in_labor_force         int64
year                                 int64
est_occ                              int64
est_own_occ                          int64
est_own_occ_2020_plus                int64
est_own_occ_2010_19                  int64
est_own_occ_2000_09                  int64
est_own_occ_1990_99                  int64
est_own_occ_1980_89                  int64
est_own_occ_1970_79                  int64
est_own_occ_1960_69                  int64
est_own_occ_1950_59                  int64
est_own_occ

In [17]:
mypasswd = getpass.getpass("Enter your database password: ")

Enter your database password: ········


In [ ]:
# Database connection details
database = "f24t03"
user = "jsmm8"
password = mypasswd
host = "pgsql"

# Create a connection to the database using SQLAlchemy for easier inserts
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{database}')

# Connect and create/drop the table, then insert data in chunks
try:
    # Connect to the database
    conn = psycopg2.connect(database=database, user=user, host=host, password=password)
    cursor = conn.cursor()
    print("Connected to the database.")

    # Drop the ethnicity table if it exists
    cursor.execute("DROP TABLE IF EXISTS merged_new_house_data;")
    print("Dropped the merged_new_house_data table if it existed.")

    # Create the <Table Name> table
    create_table_query = """
    CREATE TABLE merged_new_house_data (
    geo_id VARCHAR(255),
    block_group VARCHAR(255),
    census_tract VARCHAR(255),
    county VARCHAR(255),
    state VARCHAR(255),
    est_pop_16_and_above INT,
    est_tot_in_labor_force INT,
    est_tot_civilian INT,
    est_tot_civilian_labor_employed INT,
    est_tot_civilian_unemployed INT,
    est_tot_armed_forces INT,
    est_total_not_in_labor_force INT,
    year INT,
    est_occ INT,
    est_own_occ INT,
    est_own_occ_2020_plus INT,
    est_own_occ_2010_19 INT,
    est_own_occ_2000_09 INT,
    est_own_occ_1990_99 INT,
    est_own_occ_1980_89 INT,
    est_own_occ_1970_79 INT,
    est_own_occ_1960_69 INT,
    est_own_occ_1950_59 INT,
    est_own_occ_1940_49 INT,
    est_own_occ_1939_plus INT,
    est_own_occ_0br INT,
    est_own_occ_1br INT,
    est_own_occ_2br INT,
    est_own_occ_3br INT,
    est_own_occ_4br INT,
    est_own_occ_5br_plus INT,
    est_own_occ_kit_complete INT,
    est_own_occ_kit_lack INT,
    est_own_occ_plumb_complete INT,
    est_own_occ_plumb_lack INT,
    est_median_rooms_total INT,
    est_median_rooms_owner INT,
    est_mortgage_debt INT,
    est_without_mortgage INT,
    est_multiple_mortgages INT,
    est_total INT,
    est_under_50 INT,
    est_50_to_99 INT,
    est_100_to_124 INT,
    est_125_to_149 INT,
    est_150_to_184 INT,
    est_185_to_199 INT,
    est_200_plus INT,
    PRIMARY KEY (geo_id, year)
    );
        """
    cursor.execute(create_table_query)
    conn.commit()
    print("Created the merged_new_house_data table.")

    # Insert data from <dataframe> in chunks
    chunk_size = 500
    for start in range(0, len(merged_new_house_data), chunk_size):
        chunk = merged_new_house_data.iloc[start:start + chunk_size]
        chunk.to_sql('merged_new_house_data', engine, if_exists='append', index=False)
        print(f"Uploaded a chunk of {len(chunk)} records to the merged_new_house_data table.")

    print("Data uploaded successfully.")

    # Query the inserted data to confirm
    query = "SELECT * FROM merged_new_house_data;"
    merged_new_house_data = pd.read_sql(query, conn)
    print("Data extracted successfully! DataFrame shape:", merged_new_house_data.shape)

except Exception as e:
    print("An error occurred:", e)

finally:
    # Ensure that the cursor and connection are closed
    if cursor:
        cursor.close()
    if conn:
        conn.close()

Connected to the database.
Dropped the merged_new_house_data table if it existed.
Created the merged_new_house_data table.
Uploaded a chunk of 500 records to the merged_new_house_data table.
Uploaded a chunk of 500 records to the merged_new_house_data table.
Uploaded a chunk of 500 records to the merged_new_house_data table.
Uploaded a chunk of 500 records to the merged_new_house_data table.
Uploaded a chunk of 500 records to the merged_new_house_data table.
Uploaded a chunk of 500 records to the merged_new_house_data table.
Uploaded a chunk of 500 records to the merged_new_house_data table.
Uploaded a chunk of 500 records to the merged_new_house_data table.
Uploaded a chunk of 500 records to the merged_new_house_data table.
Uploaded a chunk of 500 records to the merged_new_house_data table.
Uploaded a chunk of 500 records to the merged_new_house_data table.
Uploaded a chunk of 500 records to the merged_new_house_data table.
Uploaded a chunk of 500 records to the merged_new_house_data 

In [ ]:
# SQL query to grant privileges
grant_privileges_query = "GRANT ALL PRIVILEGES ON TABLE merged_new_house_data TO ypd5yb, jsmm8, remcmf, jsmm8;"

with engine.connect() as connection:
        connection.execute(text(grant_privileges_query))
        print("Privileges granted successfully.")

Privileges granted successfully.
